<a href="https://colab.research.google.com/github/hiteshsurya17/hiteshsurya17-INFO-5731-Section-020---Computational-Methods-for-Information-Systems-Fall-2024-1-/blob/main/Hitesh_chowdary_suryadevara_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [14]:
#(2)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

movie_IMDB_IDs = [
    "tt9018736",  # Avatar: The Last Airbender
    "tt2631186"    # Baahubali: The Beginning
]

reviews_needed = 1000

def scrape_reviews(imdb_id, reviews_needed):
    headers = {"User-Agent": "Mozilla/5.0"}
    base_url = f"https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ql_3"
    reviews = []
    current_page = base_url

    while len(reviews) < reviews_needed:
        response = requests.get(current_page, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        review_elements = soup.find_all("div", class_="text show-more__control")
        for review in review_elements:
            if len(reviews) < reviews_needed:
                reviews.append(review.get_text())
            else:
                break

        load_more_button = soup.find('div', class_='load-more-data')
        if load_more_button:
            key = load_more_button.get('data-key')
            if key:
                next_page_url = f"https://www.imdb.com/title/{imdb_id}/reviews/_ajax?ref_=undefined&paginationKey={key}"
                current_page = next_page_url
                time.sleep(1)
            else:
                break
        else:
            break

    return reviews

reviews = []
for imdb_id in movie_IMDB_IDs:
    reviews = scrape_reviews(imdb_id, reviews_needed - len(reviews))
    reviews.extend(reviews)
    if len(reviews) >= reviews_needed:
        break

df = pd.DataFrame(reviews, columns=["Review"])
df.to_csv("imdb_movie_reviews.csv", index=False)

print(f"Scraped {len(reviews)} reviews and saved to imdb_movie_reviews.csv")


Scraped 2000 reviews and saved to imdb_movie_reviews.csv


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

df = pd.read_csv('/content/imdb_movie_reviews.csv')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Remove noise, such as special characters and punctuations.
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers.
    text = re.sub(r'\d+', '', text)

    # Remove stopwords.
    tokens = nltk.word_tokenize(text)
    text = ' '.join([word for word in tokens if word.lower() not in stop_words])

    # Convert text to lowercase.
    text = text.lower()

    return text

def apply_stemming(text):
    tokens = nltk.word_tokenize(text)
    return ' '.join([stemmer.stem(word) for word in tokens])

def apply_lemmatization(text):
    tokens = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in tokens])

df['Cleaned_Review_Body'] = df['Review'].apply(clean_text)
df['Stemmed_Review_Body'] = df['Cleaned_Review_Body'].apply(apply_stemming)
df['Lemmatized_Review_Body'] = df['Cleaned_Review_Body'].apply(apply_lemmatization)

df.to_csv('imdb_movie_reviews.csv', index=False)

print(df[['Review', 'Cleaned_Review_Body', 'Stemmed_Review_Body', 'Lemmatized_Review_Body']].head())


                                              Review  \
0  I understand that adapting a cartoon series sc...   
1  I'm too exhausted with these pointless live ac...   
2  At long last, Netflix's Avatar: The Last Airbe...   
3  I know Avatar: The Last Airbender is one of th...   
4  I really loved the animated Series of Avatar, ...   

                                 Cleaned_Review_Body  \
0  understand adapting cartoon series scene scene...   
1  im exhausted pointless live action remakes ani...   
2  long last netflixs avatar last airbender arriv...   
3  know avatar last airbender one popular loved a...   
4  really loved animated series avatar watched ma...   

                                 Stemmed_Review_Body  \
0  understand adapt cartoon seri scene scene live...   
1  im exhaust pointless live action remak anim cl...   
2  long last netflix avatar last airbend arriv mu...   
3  know avatar last airbend one popular love anim...   
4  realli love anim seri avatar watch mani tim

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [17]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import pandas as pd
import spacy
from collections import Counter

df = pd.read_csv('imdb_movie_reviews.csv')

nlp = spacy.load('en_core_web_sm')

pos_counter = Counter()
entity_counter = Counter()

def analyze_text(text):
    doc = nlp(text)

    # POS Tagging: count each part of speech (noun, verb, adj, adv)
    pos_tags = [token.pos_ for token in doc]
    pos_counter.update(pos_tags)

    # Dependency Parsing: Display dependency relations
    print("\nDependency Parsing Tree:")
    for token in doc:
        print(f"{token.text} --> {token.dep_} --> {token.head.text}")

    # Named Entity Recognition: Extract and count entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    entity_counter.update([ent.label_ for ent in doc.ents])

    return entities

df['entities'] = df['Review'].apply(analyze_text)

print("\nParts of Speech Count:")
print(pos_counter)

print("\nEntity Count:")
print(entity_counter)

df.to_csv('imdb_movie_reviews_with_analysis.csv', index=False)



Streaming output truncated to the last 5000 lines.
better --> acomp --> be
that --> nsubj --> 's
's --> ccomp --> needs
the --> det --> thing
only --> amod --> thing
thing --> attr --> 's
I --> nsubj --> think
think --> relcl --> thing
is --> ccomp --> think
weak --> acomp --> is
in --> prep --> is
the --> det --> series
series --> pobj --> in
and --> cc --> think
gives --> conj --> needs
you --> dative --> gives
feeling --> xcomp --> gives
that --> mark --> heard
you --> nsubj --> heard
have --> aux --> heard
already --> advmod --> heard
heard --> ccomp --> feeling
it --> dobj --> heard
many --> amod --> times
times --> npadvmod --> heard
somewhere --> advmod --> else
else --> advmod --> times
.... --> punct --> needs

Dependency Parsing Tree:
I --> nsubj --> know
do --> aux --> know
n't --> neg --> know
know --> ROOT --> know
why --> advmod --> slamming
so --> advmod --> many
many --> nsubj --> slamming
are --> aux --> slamming
slamming --> ccomp --> know
this --> det --> series
seri

In [19]:
'''example sentence : I recommend and give it a try.


*   Dependency parcing identifies the gramatical relationship between words in a sentence and orders it in a
    tree form with the one word acting as the 'head' of the tree .
recommend --> ROOT
  ├── I (nsubj)
  ├── and (cc)
  ├── give (conj)
    ├── it (dative)
    └── try (dobj)

*   Constituency parsing divides the sentence into sub phrases such as NP(noun phrase), PP(preposition phrase) ,
    VP(verb phrase) and arranges the sentence in a tree structure as shown:
(S (NP I)
  (VP recommend
    (CC and)
    (VP give
      (NP it)
      (NP a try))))

'''

"example sentence : I recommend and give it a try.\n\n\n*   Dependency parcing identifies the gramatical relationship between words in a sentence and orders it in a \n    tree form with the one word acting as the 'head' of the tree .\nrecommend --> ROOT \n  ├── I (nsubj) \n  ├── and (cc) \n  ├── give (conj) \n    ├── it (dative) \n    └── try (dobj)\n\n*   Constituency parsing divides the sentence into sub phrases such as NP(noun phrase), PP(preposition phrase) , \n    VP(verb phrase) and arranges the sentence in a tree structure as shown: \n(S (NP I) \n  (VP recommend \n    (CC and) \n    (VP give \n      (NP it) \n      (NP a try))))\n\n"

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [20]:
''' I had fun completing the assignment . I have learned how to scrape data from websites which load
data using javascript by using a data-key and much more from the data processing question aswell.
'''

' I had fun completing the assignment . I have learned how to scrape data from websites which load \ndata using javascript by using a data-key and much more from the data processing question aswell.\n'